**Importing required libraries**

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Importing dataset**

In [2]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Vendor_Data.csv')
df.head()

,Products,Avg_Price_per_unit,Cost_per_unit,Average_Profit_per_unit,Average_units_sold,Incremental_acquisition
0,Product 1,95,92,2.8,0.6,0.00323
1,Product 2,37,27,10.0,0.8,0.00106
2,Product 3,34,26,8.5,0.6,0.00328
3,Product 4,32,30,2.8,0.7,0.00000
4,Product 5,62,45,16.0,1.0,0.00079


**Initial Profit**

In [3]:
conv_rate = 0.05
initial_profit = conv_rate*(np.sum(df["Avg_Price_per_unit"]*df["Average_units_sold"]) - np.sum(df['Cost_per_unit']*df["Average_units_sold"]))
print(initial_profit)

636.2149999999995


**This function will get us the overall revenue for the given increment vector**

In [4]:
def find_rev(increment):
    price = df['Avg_Price_per_unit']*(1+increment)
    volumes = df['Average_units_sold']
    multiplier = (1-(df['Incremental_acquisition']*increment*10))
    total_multiplier = np.prod(multiplier)
    profit_wo_multiplier = conv_rate*(np.sum(price*volumes) - np.sum(volumes*df['Cost_per_unit']))
    profit_w_multiplier = profit_wo_multiplier*total_multiplier
    net_profit = np.sum(profit_w_multiplier)
    return net_profit

**This function tries to get the next best increment vector**

In [7]:
def incremental_new(initial_increments):
    initial_rev = find_rev(initial_increments)
    intermediate_rev = 0
    for i in range(1000):
        increments = initial_increments
    
        if increments[i] > -0.099:
            increments[i] = increments[i] - 0.01
            rev = find_rev(increments)
            if rev > initial_rev:
                final_increments = increments
                intermediate_rev = rev
            else :
                increments[i] = increments[i] + 0.01
                
        if increments[i] < 0.20:
            increments[i] = increments[i] + 0.01
            rev = find_rev(increments)
            if rev > max(initial_rev, intermediate_rev):
                final_increments = increments
            else:
                increments[i] = initial_increments[i] - 0.01
    
    final_increments = increments
    
    return final_increments

**This loop will get the maximum value of profit and the increment % in price**

In [8]:
# This is the initial value of the increment vector - all zeros
increment_i = np.repeat(0.0,1000)

#flag = 1 is a condition when the find_rev() function start giving same prfit
flag = 0
check = [0.0]
while flag==0:
    print(find_rev(increment_i))
    increment_iplus1 = incremental_new(increment_i)
    if (find_rev(increment_i) == max(check)):
        flag = 1
    check.append(find_rev(increment_i))
    increment_i = increment_iplus1

636.2149999999995
705.2949457154343
781.1184422560419
864.2094294666919
955.1040615795954
1054.3483379898291
1162.5265231604085
1280.2757778018886
1408.1976691979569
1546.8858525233732
1696.929464723767
1697.223571655071
1711.721767042963
1712.2365197845438
1725.9453027170437
1726.3901639002722
1742.0384321581237
1742.310643793764
1757.3476637481979
1757.8197277914924
1772.425143109625
1772.6666439926053
1787.1461810576327
1787.6288182775522
1801.5824667147322
1802.0180658488
1816.2630713894903
1816.5713149092596
1829.499771451029
1829.792365691026
1841.9262065514392
1842.149542063483
1853.7249261458137
1854.2270745889687
1865.6392031039738
1866.1092968683658
1877.6418113802204
1878.080220848973
1889.7080414367579
1890.2083021707967
1901.5957290925583
1902.0601564377703
1912.332728332717
1912.7947607544963
1923.6762240435805
1923.8954065321984
1934.9116614653292
1935.4586834545562
1946.1812561320467
1946.6564761691589
1957.2194628876603
1957.6907641717432
1968.2847004204896
1968.760461

**Adding increment % column in dataset**

In [9]:
df['%_Increment_Opt_Price'] = np.around(increment_i,2)
print(df)

         Products  Avg_Price_per_unit  Cost_per_unit  Average_Profit_per_unit  \
0       Product 1                  95             92                      2.8   
1       Product 2                  37             27                     10.0   
2       Product 3                  34             26                      8.5   
3       Product 4                  32             30                      2.8   
4       Product 5                  62             45                     16.0   
..            ...                 ...            ...                      ...   
995   Product 996                  27             26                      1.0   
996   Product 997                  94             77                     17.0   
997   Product 998                  17             13                      4.0   
998   Product 999                  31             29                      2.0   
999  Product 1000                  55             55                      0.0   

     Average_units_sold  In

**Exporting updated file in CSV format**

In [13]:
df.to_csv("Update_Vendor_Data.csv",index=False)
df.head()

,Products,Avg_Price_per_unit,Cost_per_unit,Average_Profit_per_unit,Average_units_sold,Incremental_acquisition,%_Increment_Opt_Price
0,Product 1,95,92,2.8,0.6,0.00323,-0.1
1,Product 2,37,27,10.0,0.8,0.00106,-0.1
2,Product 3,34,26,8.5,0.6,0.00328,-0.1
3,Product 4,32,30,2.8,0.7,0.00000,0.2
4,Product 5,62,45,16.0,1.0,0.00079,-0.1


**Optimized_profit:**

In [11]:
Optimum_profit = max(check)
print(np.around(Optimum_profit,2))

2980.3
